Tóm tắt quá trình chính của phần ứng dụng:
1. Load thư viện cần thiết.
2. Tải model đã train trong báo cáo và load model.
3. Tải và load video Input.
4. Tách video thành các frame.
5. Model dự đoán trên các frame và đánh label từng frame.
6. Ghép các frame đã đánh label thành video với frame rate của video gốc.

Thêm vào các thư viện cần sử dụng.

In [ ]:
import os
import cv2
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image

Tải video để mô hình dự đoán.

In [ ]:
!gdown --id 1iONnVLP-ibMB53e_JNloWpJFooYqsD6j
!gdown --id 18m2luBnJv-63D2I0kJq8Iu-t5lcGRlSV

Tải model CNN_SVM đã được train từ file báo cáo.

In [ ]:
!gdown --id 1M7jvQixVuinrWLPTpTvxHgztUBt4B5JB

Downloading...
From: https://drive.google.com/uc?id=1M7jvQixVuinrWLPTpTvxHgztUBt4B5JB
To: /content/video_model.h5
38.7MB [00:01, 30.6MB/s]


Load model.

In [ ]:
# Load the best model
try:
  model = keras.models.load_model(filepath="/content/video_model.h5")
  print("Load Model Successfully!")
except:
  print("Failed to load model! Please check again!")

Load Model Successfully!


# Dự đoán của mô hình.

Định nghĩa hàm tách video thành các frames và hàm clean (để xoá dữ liệu trong folder cũ khi test video mới).

In [ ]:
def video_to_images(inp, out):
  global FPS_
  video_cap = cv2.VideoCapture(inp)
  FPS_ = int(video_cap.get(cv2.CAP_PROP_FPS))
  success, image = video_cap.read()
  count = 0
  while success:
    video_cap.set(cv2.cv2.CAP_PROP_FPS, FPS_)
    cv2.imwrite(out + str(count) + ".jpg" , image)
    success, image = video_cap.read()
    count += 1
  video_cap.release()
  cv2.destroyAllWindows()

def clean(path_):
  for f in os.listdir(path_):
      os.remove(os.path.join(path_, f))

Thiết lập đường dẫn để lưu các frame.

In [ ]:
framePath = "/content/frame"
output = "/content/processed_frame"
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
thickness = 2

if not os.path.isdir(framePath):
  os.mkdir(framePath)
else:
  clean(framePath)

if not os.path.isdir(output):
  os.mkdir(output)
else:
  clean(output)

framePath+="/"
output+="/"

Tiến hành tách frames từ video.

In [ ]:
video_to_images("/content/non_accident_1.mp4", framePath)
frame_lst = os.listdir(framePath)

Đưa ra dự đoán trên từng frame và đánh label lên góc trái của frame.

In [ ]:
for _ in range(len(frame_lst)):
  base_ = cv2.imread(framePath + frame_lst[_])
  img_ = image.load_img(framePath + frame_lst[_], target_size = (224,224))
  img_ = image.img_to_array(img_)
  img_= img_/255
  img_ = np.expand_dims(img_, axis = 0)
  if model.predict(img_) < 0:
    color = (0,0,255)
    img_ = cv2.putText(base_, 'Status: accident', org, font, fontScale, color, thickness, cv2.LINE_AA)
  else:
    color = (124,252,0)
    img_ = cv2.putText(base_, 'Status: non-accident', org, font, fontScale, color, thickness, cv2.LINE_AA)
  cv2.imwrite(output+str(_)+".jpg",img_)

Sau khi đã thu được tập các frame được gán label. Ta tiến hành ghép các frame lại với nhau với Frame rate của video gốc (biến FPS_).

In [ ]:
pro_frame = os.listdir(output)
tmp = cv2.imread(framePath + frame_lst[0]).shape
w, h = tmp[1], tmp[0]
fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
video_ = cv2.VideoWriter('/content/predicted_video.mp4', fourcc, FPS_, (w, h))

for idx in range (len(pro_frame)):
  vid_ = cv2.imread(output+str(idx)+".jpg")
  video_.write(vid_)
print("Saved video with name: predicted_video.mp4")

cv2.destroyAllWindows()
video_.release()

Saved video with name: predicted_video.mp4
